In [0]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
# !tar xf spark-2.4.0-bin-hadoop2.7.tgz
# !pip install -q findspark


In [5]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
# !pip install -q findspark

import findspark
findspark.init('/Users/vamshi/Documents/spark-3.0.0-preview2-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
# spark = SparkSession.builder.master("local[*]")\
#         .config("spark.executor.memory", "16g")\
#         .config("spark.driver.memory", "16g")\
#         .config("spark.memory.offHeap.enabled",'true')\
#         .config("spark.memory.offHeap.size","16g")\
#         .getOrCreate()


# !ls

DIC_PySpark_Part1.ipynb Untitled3.ipynb         spark-warehouse
Untitled.ipynb          Untitled4.ipynb         test.csv
Untitled1.ipynb         mapping.csv             train.csv
Untitled2.ipynb         sample.csv


In [6]:
import numpy as np
import pandas as pd
from ast import literal_eval 
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import regexp_replace,col,array_contains,explode
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from functools import reduce
from pyspark.sql.functions import concat_ws
from pyspark.ml.classification import LogisticRegression

In [40]:
pd_df = pd.read_csv(r'train.csv')
data_spark_df = spark.createDataFrame(pd_df)
pd_df['genre']= pd_df['genre'].apply(literal_eval)
all_genre = pd_df['genre'].to_list()
# names =['Drama','Comedy','Romance Film','Thriller','Action','World cinema','Crime Fiction','Horror','Black-and-white','Indie','Action/Adventure','Adventure','Family Film','Short Film','Romantic drama','Animation','Musical','Science Fiction','Mystery','Romantic comedy']

df_mapping = pd.read_csv("mapping.csv")
dfspark_mapping = spark.createDataFrame(df_mapping)

genres = []
for i in range(dfspark_mapping.count()):
    genres.append(dfspark_mapping.take(20)[i][1])

matrix = np.zeros((len(pd_df),len(genres)))
for i,genre in enumerate(all_genre):
  for j,g in enumerate(genre):
    for k,name in enumerate(genres):
        if name==g:
          matrix[i][k] = 1
names = "Drama , Comedy , Romance Film , Thriller , Action , World cinema , Crime Fiction , Horror , Black-and-white , Indie , Action/Adventure , Adventure , Family Film , Short Film , Romantic drama , Animation , Musical , Science Fiction , Mystery , Romantic comedy"
np.savetxt("genre_lables.csv", matrix, delimiter=",",fmt='%d',header=names)
matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [8]:
lables_df = pd.read_csv(r'genre_lables.csv')
test_pd_df = pd.read_csv(r'test.csv')
lables_spark_df = spark.createDataFrame(lables_df)
test_spark_df = spark.createDataFrame(test_pd_df)

In [9]:
ddf1 = data_spark_df.withColumn("row_id", monotonically_increasing_id())
ddf2 = lables_spark_df.withColumn("row_id", monotonically_increasing_id())
ddf3 = test_spark_df.withColumn("row_id", monotonically_increasing_id())
df = ddf1.join(ddf2, "row_id").drop("row_id")
test_df = ddf3.join(ddf2, "row_id").drop("row_id")


In [10]:
%%time
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
hashingTF = HashingTF(inputCol="filtered", outputCol="features")
pipeline = Pipeline(stages=[regexTokenizer, remover, hashingTF])

model = pipeline.fit(df)
dataset = model.transform(df)

model2 = pipeline.fit(test_df)
test_dataset = model2.transform(test_df)

CPU times: user 69.7 ms, sys: 14.9 ms, total: 84.6 ms
Wall time: 814 ms


In [0]:
# train_dataset, validation_dataset = dataset.randomSplit([1.0, 0.0], seed = 123)

In [12]:
%%time
dfList = []
labelCols = lables_spark_df.columns
lr = LogisticRegression(featuresCol = 'features',maxIter=1)
# lr = LogisticRegression(featuresCol = 'features',maxIter=50)
for labelCol in labelCols:
    lr.setLabelCol(labelCol)
    # lrModel = lr.fit(train_dataset)
    lrModel = lr.fit(dataset)
    predictions = lrModel.transform(test_dataset)
    # predictions = predictions.drop("features",labelCol,"rawPrediction","probability")
    predictions = predictions.withColumn("prediction",F.col("prediction").cast(IntegerType()))
    dfList.append(predictions.select("movie_id","prediction"))

CPU times: user 2.13 s, sys: 822 ms, total: 2.95 s
Wall time: 7min 30s


In [13]:
%%time
dfs_renamed = [df.selectExpr('movie_id', f'prediction as prediction_{i}') for i, df in enumerate(dfList)]
temp_df = reduce(lambda x, y: x.join(y, ['movie_id'], how='full'), dfs_renamed)
col_list = ['prediction_%d' % i for i in range(len(dfList))]
temp_df = temp_df.withColumn('predictions',concat_ws(" ",*col_list)).drop(*col_list).toPandas().to_csv("predictions_part1.csv",index=False)

CPU times: user 932 ms, sys: 447 ms, total: 1.38 s
Wall time: 8min 19s


In [45]:
names = []
for i in range(dfspark_mapping.count()):
    names.append(dfspark_mapping.take(20)[i][1])

In [47]:
names

['Drama',
 'Comedy',
 'Romance Film',
 'Thriller',
 'Action',
 'World cinema',
 'Crime Fiction',
 'Horror',
 'Black-and-white',
 'Indie',
 'Action/Adventure',
 'Adventure',
 'Family Film',
 'Short Film',
 'Romantic drama',
 'Animation',
 'Musical',
 'Science Fiction',
 'Mystery',
 'Romantic comedy']